In [124]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [131]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([5., 5.]), afMean = tensor(5e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

loat64)
p.sum tensor(0.0004, dtype=torch.float64)
p.sum tensor(0.0005, dtype=torch.float64)
p.sum tensor(0.0007, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0005, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0005, dtype=torch.float64)
p.sum tensor(0.0008, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0004, dtype=torch.float64)
p.sum tensor(0.0005, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0009, dtype=torch.float64)
p.sum tensor(0.0015, dtype=torch.float64)
p.sum tensor(0.0004, dtype=torch.float64)
p.sum tensor(0.0010, dtype=torch.float64)
p.sum tensor(0.0008, dtype=torch.float64)
p.sum tensor(0.0006, dtype=torch.float64)
p.sum tensor(0.0004, dtype=torch.float64)
p.sum tensor(0.0005, dtype=torch.float64)
p.sum tensor(0.0004, dtype=torch.float64)
p.sum tensor(0.0007, dtype=torch.float64)
p.sum tensor(0.0004, dtype

In [140]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Categorical, Uniform
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp

def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


nHypotheses = 4
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)

# TODO: do this in numpy natively
pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAllnp = [1 - samplePDs.sum(), *samplePDs]
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])
print(pdsAllShaped)
print(pdsAll)
alpha = .05
def model(data):
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        #rr = numpyro.sample("rr", Normal(0, 1))
        
        probs = numpyro.sample("probs", Dirichlet(np.ones(kConditions)))
        # just slows things down probs = numpyro.deterministic("probs_scaled", probs * pdsAll)

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

pdsAll [0.59523809 0.17857143 0.17857143 0.04761905]
[[0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]]
[0.5952381  0.17857143 0.17857143 0.04761905]


In [141]:
nuts = NUTS(model)
init_rng_key = random.PRNGKey(1273)
mcmc = MCMC(nuts, 100, 1000)
res = mcmc.run(init_rng_key, altCounts)
mcmc.print_summary()

sample: 100%|██████████| 1100/1100 [07:37<00:00,  2.41it/s, 255 steps of size 2.67e-03. acc. prob=0.80]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.05      0.00      0.05      0.05      0.05   1635.41      1.00
   beta[1]      0.11      0.00      0.11      0.11      0.11   1495.92      1.00
   beta[2]      0.88      0.00      0.88      0.88      0.89    872.50      1.00
probs[0,0]      0.20      0.00      0.20      0.20      0.21   1267.75      1.00
probs[0,1]      0.35      0.00      0.35      0.34      0.35   1050.52      1.00
probs[0,2]      0.35      0.00      0.35      0.34      0.35   1023.12      1.00
probs[0,3]      0.10      0.00      0.10      0.10      0.10   1163.84      1.00
probs[1,0]      0.29      0.00      0.29      0.29      0.29   1028.57      1.00
probs[1,1]      0.09      0.00      0.09      0.09      0.10    891.92      1.00
probs[1,2]      0.48      0.00      0.48      0.48      0.49    507.59      1.00
prob

In [142]:
# Attempt at scaling using deterministic
# EXACTLY the same results! but much slower to converge
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00128553 0.00142994 0.00138522 0.00089227]
 [0.00117365 0.00072433 0.00132738 0.0008557 ]
 [0.00062509 0.00052084 0.00050526 0.00029141]
 [0.00117701 0.00127759 0.00074707 0.00087778]]
[[0.20408821 0.34632066 0.34686548 0.10272554]
 [0.29232916 0.09458447 0.48488292 0.12820342]
 [0.5951777  0.17888209 0.17818089 0.04775922]
 [0.29560748 0.48144063 0.09460145 0.1283504 ]]


In [137]:
# No scaling by pDs
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00135276 0.00143652 0.0015077  0.00094736]
 [0.00116426 0.00074751 0.00128939 0.0008272 ]
 [0.00062613 0.0005143  0.00049686 0.00026963]
 [0.00121846 0.00132125 0.00071697 0.00086843]]
[[0.20408753 0.3462562  0.3469338  0.10272242]
 [0.2923729  0.09458682 0.48486227 0.12817791]
 [0.595193   0.17888129 0.17816997 0.04775575]
 [0.29565826 0.4814778  0.09456076 0.12830308]]


In [134]:
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00112425 0.00072443 0.00122722 0.0008287 ]
 [0.00129735 0.00151242 0.001484   0.00094055]
 [0.00062192 0.00050019 0.00049377 0.00026432]
 [0.00126624 0.00133723 0.00076884 0.00085534]]
[[0.29231083 0.0946319  0.48484075 0.12821656]
 [0.20410876 0.34625798 0.34691718 0.10271599]
 [0.5951638  0.17888646 0.17819968 0.04774999]
 [0.2955718  0.48146    0.09461594 0.12835221]]


In [122]:
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00130185 0.00144853 0.00142383 0.00093716]
 [0.00119027 0.00075693 0.00129492 0.00086173]
 [0.00063651 0.00052852 0.00050821 0.0002702 ]
 [0.00122513 0.00136455 0.00075551 0.00086868]]
[[0.201412   0.34808743 0.34775582 0.10274476]
 [0.29363525 0.09596074 0.48135352 0.12905048]
 [0.5961557  0.17827164 0.17761181 0.04796095]
 [0.2937598  0.48405725 0.09401795 0.12816498]]


In [126]:
s.mean(0) * pdsAll 

DeviceArray([[0.1198881 , 0.06215847, 0.06209926, 0.00489261],
             [0.17478289, 0.01713585, 0.08595598, 0.00614526],
             [0.35485458, 0.03183422, 0.0317164 , 0.00228385],
             [0.17485702, 0.0864388 , 0.01678892, 0.00610309]],            dtype=float32)

In [127]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([2.7715e-04, 4.5567e-04, 8.9430e-05, 1.2151e-04], dtype=torch.float64)

In [130]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0) 

tensor([2.7715e-04, 4.5567e-04, 8.9430e-05, 1.2151e-04], dtype=torch.float64)

In [105]:
pdsAllnp

[0.5952380895614624, 0.17857143, 0.17857143, 0.04761905]

In [106]:
liabParams55cov["afMean"]

tensor(0.0005)

In [107]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([2.7699e-04, 4.5726e-04, 8.9407e-05, 1.2194e-04], dtype=torch.float64)